# RTM Simulation: Vascular Tree (Fractal Lattice)

This notebook demonstrates the **fractal vascular regime** in the RTM framework.

## Theoretical Background

### Biological Vascular Networks

Biological vasculature (blood vessels, bronchi, neural trees) exhibits:
- **Hierarchical branching** (Murray's law)
- **Geometric scaling** of vessel diameter with depth
- **Fractal self-similarity** across scales

### RTM Predictions

For fractal biological networks, RTM predicts:

$$\alpha \approx 2.5$$

This is:
- Higher than diffusive (α = 2) due to hierarchical bottlenecks
- Lower than quantum systems (α ≈ 3.5) due to evolutionary optimization

**Expected result:** α ≈ 2.5 ± 0.03

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
from collections import deque

RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

# Tree parameters
BRANCHING = 3
SCALE_FACTOR = 0.7

print("Libraries loaded!")

## Vascular Tree Class

In [ ]:
class VascularTree:
    def __init__(self, max_gen, branching=3, scale=0.7, rng=None):
        self.max_gen = max_gen
        self.branching = branching
        self.scale = scale
        self.rng = rng or np.random.default_rng()
        self.adj = {}
        self.positions = {}
        self.node_gen = {}
        self.root = 0
        self.leaves = []
        self._build()
        
    def _random_dir(self):
        theta = self.rng.uniform(0, 2*np.pi)
        phi = np.arccos(self.rng.uniform(-1, 1))
        return np.array([np.sin(phi)*np.cos(theta), np.sin(phi)*np.sin(theta), np.cos(phi)])
    
    def _build(self):
        self.positions[0] = np.array([0., 0., 0.])
        self.adj[0] = []
        self.node_gen[0] = 0
        next_id = 1
        queue = deque([(0, 0, np.array([0,0,1]))])
        
        while queue:
            parent, gen, pdir = queue.popleft()
            if gen >= self.max_gen:
                self.leaves.append(parent)
                continue
            seg_len = self.scale ** gen
            for _ in range(self.branching):
                child = next_id
                next_id += 1
                direction = 0.3*pdir + 0.7*self._random_dir()
                direction /= np.linalg.norm(direction)
                self.positions[child] = self.positions[parent] + seg_len * direction
                self.adj[child] = []
                self.node_gen[child] = gen + 1
                self.adj[parent].append(child)
                self.adj[child].append(parent)
                queue.append((child, gen+1, direction))
    
    @property
    def n_nodes(self): return len(self.adj)
    
    @property
    def effective_depth(self):
        return (1 - self.scale**self.max_gen) / (1 - self.scale)

print("VascularTree class defined.")

## Visualize Tree Structure

In [ ]:
tree = VascularTree(4, rng=np.random.default_rng(123))

fig, ax = plt.subplots(figsize=(10, 8))

for node, neighbors in tree.adj.items():
    x1, y1, _ = tree.positions[node]
    for neighbor in neighbors:
        if neighbor > node:
            x2, y2, _ = tree.positions[neighbor]
            gen = tree.node_gen[node]
            ax.plot([x1, x2], [y1, y2], '-', color=plt.cm.Greens(0.3 + 0.5*gen/4),
                   linewidth=max(0.5, 3-gen*0.5))

ax.scatter([0], [0], s=200, c='red', marker='*', zorder=10, label='Root')
for leaf in tree.leaves:
    x, y, _ = tree.positions[leaf]
    ax.scatter([x], [y], s=20, c='darkgreen', zorder=5)

ax.set_aspect('equal')
ax.set_title(f'Vascular Tree (gen=4, {tree.n_nodes} nodes)', fontsize=14)
ax.legend()
plt.show()

## Run Simulation

In [ ]:
def random_walk_to_leaf(adj, root, leaves, max_steps, rng):
    current = root
    leaves_set = set(leaves)
    for step in range(max_steps):
        if current in leaves_set:
            return step
        neighbors = adj[current]
        if not neighbors:
            return -1
        current = rng.choice(neighbors)
    return -1

GENERATIONS = [2, 3, 4, 5, 6]
N_REAL = 10
N_WALKS = 50
MAX_STEPS = 1_000_000

results = []
print(f"{'Gen':>4} | {'N':>6} | {'L_eff':>8} | {'T_mean':>12}")
print("-" * 40)

for gen in GENERATIONS:
    all_mfpt = []
    sample_tree = VascularTree(gen, rng=rng)
    
    for _ in range(N_REAL):
        tree = VascularTree(gen, rng=rng)
        for _ in range(N_WALKS):
            ht = random_walk_to_leaf(tree.adj, tree.root, tree.leaves, MAX_STEPS, rng)
            if ht >= 0:
                all_mfpt.append(ht)
    
    T_mean = np.mean(all_mfpt)
    L_eff = sample_tree.effective_depth
    results.append({'gen': gen, 'N': sample_tree.n_nodes, 'L': L_eff, 'T': T_mean})
    print(f"{gen:>4} | {sample_tree.n_nodes:>6} | {L_eff:>8.3f} | {T_mean:>12.2f}")

df = pd.DataFrame(results)

## Fit Power Law

In [ ]:
log_L = np.log10(df['L'].values)
log_T = np.log10(df['T'].values)

slope, intercept, r_value, _, std_err = stats.linregress(log_L, log_T)

print("=" * 50)
print("POWER LAW FIT")
print("=" * 50)
print(f"\nFitted α = {slope:.4f} ± {std_err:.4f}")
print(f"R² = {r_value**2:.6f}")
print(f"\nRTM prediction: α ≈ 2.5")
print(f"Paper reported: α ≈ 2.5 ± 0.03")

## Visualization

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

ax.loglog(df['L'], df['T'], 'o', markersize=12, color='darkgreen', label='Simulation')

L_fit = np.linspace(df['L'].min()*0.8, df['L'].max()*1.2, 100)
T_fit = 10**intercept * L_fit**slope
ax.plot(L_fit, T_fit, 'r-', linewidth=2, label=f'Fit: α = {slope:.3f}')

T_25 = 10**intercept * L_fit**2.5
ax.plot(L_fit, T_25, 'orange', linestyle='--', linewidth=1.5, alpha=0.7, label='α = 2.5 (RTM)')

ax.set_xlabel('Effective Depth L', fontsize=14)
ax.set_ylabel('MFPT', fontsize=14)
ax.set_title(f'Vascular Tree: α = {slope:.3f}', fontsize=16)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3, which='both')
plt.show()

## Summary

In [ ]:
print(f"""
SUMMARY: Vascular Tree Simulation
==================================

RTM Prediction: α ≈ 2.4 – 2.6
Paper Reported: α ≈ 2.5 ± 0.03

This Simulation: α = {slope:.4f} ± {std_err:.4f}

INTERPRETATION:
The vascular tree models biological transport networks:
  • Blood vessels
  • Bronchial trees
  • Neural dendrites

The α ≈ 2.5 reflects:
  • Hierarchical branching structure
  • Evolutionary optimization for transport
  • Fractal self-similarity

STATUS: {'✓ CONFIRMED' if abs(slope - 2.5) < 0.2 else '⚠ NEEDS REVIEW'}
""")